In [ ]:
# !pip install earthengine-api
# !pip install altair
# !pip install geemap

In [325]:
# Import the required libraries

import pandas as pd
import altair as alt
import numpy as np
import folium
import ee
import geemap
from tqdm import tqdm

<p> We are using google earth engine api to pull and analize the data from the google earthengine server. For that, we first need to authenticate the earth engine api with proper google credentials. to know more about the authentication process checkout <a href="https://developers.google.com/earth-engine/apidocs/ee-authenticate"><b>Authenticator</b></a>


In [300]:
ee.Authenticate()
ee.Initialize()

Enter verification code: 4/1AfgeXvt_wGmV5e0u3JfqWgvf9KfgOLrLYnOCTRvL-ySnpg2-cptQEnsthxU

Successfully saved authorization token.


Now we will define our analysis time frame. We are considering images of 20 years earlier from the current timestamp.  

In [326]:
today = ee.Date(pd.to_datetime('today'))
date_range = ee.DateRange(today.advance(-20, 'years'), today)

The are of interest is the <a href="https://en.wikipedia.org/wiki/Kutupalong_refugee_camp"><b>Kutupalong Rohingya Camp</b></a>, Cox's Bazar, in Bangladesh. 

In [343]:
aoi_shp_path =  "aoi/Kutupalong-AOI-Polygon-polygon.shp"
aoi = geemap.shp_to_ee(aoi_shp_path)
aoi

In [344]:
aoi.getInfo()

{'type': 'FeatureCollection',
 'columns': {'Name': 'String',
  'descriptio': 'String',
  'system:index': 'String',
  'tessellate': 'Integer'},
 'features': [{'type': 'Feature',
   'geometry': {'geodesic': False,
    'type': 'Polygon',
    'coordinates': [[[92.12844545199196, 21.17519523146724],
      [92.12488402839178, 21.2198086869558],
      [92.1774510964344, 21.21985803329638],
      [92.17108354208149, 21.17496411425513],
      [92.12844545199196, 21.17519523146724]]]},
   'id': '0',
   'properties': {'Name': 'Kutupalong-AOI-Polygon',
    'descriptio': '',
    'tessellate': 1}}]}

In [345]:
import numpy as np
aoi_geometry = aoi.geometry()
coords = aoi_geometry.getInfo()['coordinates']
mapBoundary = coords[0][1]

coords = np.array(coords[0])
coords

array([[92.12844545, 21.17519523],
       [92.12488403, 21.21980869],
       [92.1774511 , 21.21985803],
       [92.17108354, 21.17496411],
       [92.12844545, 21.17519523]])

In [346]:
landsat =  ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA')\
.filterMetadata('WRS_PATH','equals',136).filterMetadata('WRS_ROW', 'equals',45)\
.filterDate(date_range)\
.filter(ee.Filter.lt('CLOUD_COVER', 20))
print("Total Image Downloaded: ",landsat.size().getInfo()) #gets the total number of images

Total Image Downloaded:  76


In [347]:
test_image = landsat.first()
test_image = test_image.clip(aoi)

In [306]:
test_image.getInfo()

{'type': 'Image',
 'bands': [{'id': 'B1',
   'data_type': {'type': 'PixelType', 'precision': 'float'},
   'dimensions': [7661, 7811],
   'crs': 'EPSG:32646',
   'crs_transform': [30, 0, 239085, 0, -30, 2514015]},
  {'id': 'B2',
   'data_type': {'type': 'PixelType', 'precision': 'float'},
   'dimensions': [7661, 7811],
   'crs': 'EPSG:32646',
   'crs_transform': [30, 0, 239085, 0, -30, 2514015]},
  {'id': 'B3',
   'data_type': {'type': 'PixelType', 'precision': 'float'},
   'dimensions': [7661, 7811],
   'crs': 'EPSG:32646',
   'crs_transform': [30, 0, 239085, 0, -30, 2514015]},
  {'id': 'B4',
   'data_type': {'type': 'PixelType', 'precision': 'float'},
   'dimensions': [7661, 7811],
   'crs': 'EPSG:32646',
   'crs_transform': [30, 0, 239085, 0, -30, 2514015]},
  {'id': 'B5',
   'data_type': {'type': 'PixelType', 'precision': 'float'},
   'dimensions': [7661, 7811],
   'crs': 'EPSG:32646',
   'crs_transform': [30, 0, 239085, 0, -30, 2514015]},
  {'id': 'B6',
   'data_type': {'type': 'Pi

In [350]:
# ##### earth-engine drawing method setup
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
      tiles = map_id_dict['tile_fetcher'].url_format,
      attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
      name = name,
      overlay = True,
      control = True
  ).add_to(self)

# configuring earth engine display rendering method in folium
folium.Map.add_ee_layer = add_ee_layer

# visual parameters for the satellite imagery natural colors display
image_params = {
  'bands': ['B4',  'B3',  'B2'],
  'min': 0,
  'max': 0.3,
  'gamma': 1
}

map = folium.Map(location=[coords[:,1].mean(), coords[:,0].mean()], 
                 zoom_start=13, height=400, control_scale=True)
map.add_ee_layer(test_image, image_params, 'LandsatFirst')
map.add_child(folium.LayerControl())
map

In [351]:
# NDBI = (SWIR - NIR) / (SWIR + NIR)

In [352]:
nir  = test_image.select('B4')
swir = test_image.select('B5')


ndbi = swir.subtract(nir).divide(swir.add(nir)).rename('NDBI')

In [353]:
ndbi_visParams = {min: -1, max: 1};


map_ndbi = folium.Map(location=[coords[:,1].mean(), coords[:,0].mean()], 
                 zoom_start=13, height=400, control_scale=True)
map_ndbi.add_ee_layer(ndbi, ndbi_visParams, 'NDBI')

map_ndbi.add_child(folium.LayerControl())


map_ndbi

In [354]:
# doi.toList(doi.size())
landsat.size()

In [355]:
def get_ndbi_info(img):
    
    img_date = img.date().getInfo()['value']
    img_sytemindex = img.get('system:index').getInfo()
  
    nir  = img.select('B4')
    swir = img.select('B5')
    
    ndbi = swir.subtract(nir).divide(swir.add(nir)).rename('NDBI')
    
    min_ndbi_val = ndbi.reduceRegion(ee.Reducer.min(), aoi, 30,  crs = 'EPSG:32648', bestEffort = True, maxPixels = 1e9).getInfo()['NDBI']
    max_ndbi_val = ndbi.reduceRegion(ee.Reducer.max(), aoi, 30,  crs = 'EPSG:32648', bestEffort = True, maxPixels = 1e9).getInfo()['NDBI']
    mean_ndbi_val = ndbi.reduceRegion(ee.Reducer.mean(), aoi, 30,  crs = 'EPSG:32648', bestEffort = True, maxPixels = 1e9).getInfo()['NDBI']
    
    img_all_info = {
      'system:index': img_sytemindex,
      'date': img_date,
      'min_ndbi' : min_ndbi_val,
      'max_ndbi' : max_ndbi_val,
      'mean_ndbi' : mean_ndbi_val      
      }

    return img_all_info


In [356]:
landsat

In [357]:
landsat_list = landsat.toList(landsat.size())
landsat_list_size = landsat_list.size().getInfo()

import pandas as pd
df = pd.DataFrame(columns=['SystemIndex', 'Millis', 'MinNDBI', 'MaxNDBI', 'MeanNDBI'])

for i in tqdm(range(landsat_list_size)):
    image = ee.Image(landsat_list.get(i))
    image_info = get_ndbi_info(image)
#     print(image_info)
    df = df.append({'SystemIndex': image_info['system:index'], 
                    'Millis':  image_info['date'], 
                    'MinNDBI': image_info['min_ndbi'], 
                    'MaxNDBI': image_info['max_ndbi'], 
                    'MeanNDBI': image_info['mean_ndbi']}, 
                    ignore_index=True)

100%|██████████████████████████████████████████████████████████████████████████████████| 76/76 [04:54<00:00,  3.88s/it]


In [358]:
df

,SystemIndex,Millis,MinNDBI,MaxNDBI,MeanNDBI
0,LC08_136045_20130912,1378959662280,-0.501476,0.856036,0.691952
1,LC08_136045_20131014,1381724454570,-0.293474,0.832426,0.642996
2,LC08_136045_20131030,1383106845650,-0.365039,0.849545,0.705365
3,LC08_136045_20131201,1385871639720,-0.326832,0.789117,0.608659
4,LC08_136045_20131217,1387254032730,-0.41801,0.794675,0.586202
...,...,...,...,...,...
71,LC08_136045_20210427,1619497117542,0.022426,0.436299,0.156
72,LC08_136045_20210529,1622261925631,-0.02558,0.636801,0.238946
73,LC08_136045_20211105,1636085967127,-0.142485,0.791913,0.487977
74,LC08_136045_20211121,1637468362754,-0.202247,0.766042,0.448745


In [359]:
def add_date_info(df):
  df['Timestamp'] = pd.to_datetime(df['Millis'], unit='ms')
  df['Year'] = pd.DatetimeIndex(df['Timestamp']).year
  df['Month'] = pd.DatetimeIndex(df['Timestamp']).month
  df['Day'] = pd.DatetimeIndex(df['Timestamp']).day
  df['DOY'] = pd.DatetimeIndex(df['Timestamp']).dayofyear
  return df

In [360]:
df = add_date_info(df).drop(columns=['Millis'])
df

,SystemIndex,MinNDBI,MaxNDBI,MeanNDBI,Timestamp,Year,Month,Day,DOY
0,LC08_136045_20130912,-0.501476,0.856036,0.691952,2013-09-12 04:21:02.280,2013,9,12,255
1,LC08_136045_20131014,-0.293474,0.832426,0.642996,2013-10-14 04:20:54.570,2013,10,14,287
2,LC08_136045_20131030,-0.365039,0.849545,0.705365,2013-10-30 04:20:45.650,2013,10,30,303
3,LC08_136045_20131201,-0.326832,0.789117,0.608659,2013-12-01 04:20:39.720,2013,12,1,335
4,LC08_136045_20131217,-0.41801,0.794675,0.586202,2013-12-17 04:20:32.730,2013,12,17,351
...,...,...,...,...,...,...,...,...,...
71,LC08_136045_20210427,0.022426,0.436299,0.156,2021-04-27 04:18:37.542,2021,4,27,117
72,LC08_136045_20210529,-0.02558,0.636801,0.238946,2021-05-29 04:18:45.631,2021,5,29,149
73,LC08_136045_20211105,-0.142485,0.791913,0.487977,2021-11-05 04:19:27.127,2021,11,5,309
74,LC08_136045_20211121,-0.202247,0.766042,0.448745,2021-11-21 04:19:22.754,2021,11,21,325


In [361]:
alt.Chart(df).transform_fold(
    ['MinNDBI', 'MaxNDBI', 'MeanNDBI'],
    as_=['NDBI_Type', 'Value']
).mark_line().encode(
    x='Timestamp:T',
    y='Value:Q',
    color='NDBI_Type:N'
)

alt.Chart(...)